In [1]:
import os
os.chdir("../src/")

In [2]:
import import_inpi as inpi
import transform_rcs as trcs
import pandas as pd

In [ ]:
inpi.import_inpi()

In [3]:
df_rep_2018, df_pm_2018  = trcs.transform_rcs("2018")

Taille des tables
7
8
taille des tables
(1142, 49)
(670, 29)


In [4]:
df_rep_2019, df_pm_2019 = trcs.transform_rcs("2019")

/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (39,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (7,14,15,17,24,25,27,28,29,30,31,39,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (17,32,35,39,40,42) have mixed types. Spec

Taille des tables
668
668
taille des tables
(729802, 48)
(289656, 29)


In [5]:
df_rep_2020, df_pm_2020  = trcs.transform_rcs("2020")

/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (25,26,33,39,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (7,17,32,34) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (15,32,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (7,34) have mixed types. Specify dtype option on import or set low_memory=False.
  datainpi = pd.read_csv(zip_file.open(myfile), sep=";")
/home/coder/work/data_inpi/src/transform_rcs.py:22: DtypeWarning: Columns (15,32,35,37) have mixed types. Specify dtype option

Taille des tables
544
544
taille des tables
(907035, 49)
(414967, 29)


In [7]:
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import requests
from zipfile import ZipFile

def isDirectory(url):
    if(url.endswith('/') and ".." not in url):
        return True
    else:
        return False
list_urls = []
def findLinks(url):
    page = requests.get(url).content
    bsObj = BeautifulSoup(page, 'html.parser')
    maybe_directories = bsObj.findAll('a', href=True)
    list_df = []
    
    for link in maybe_directories:
        if(isDirectory(link['href'])):
            newUrl = url + link['href']   
            print(newUrl)   
            list_urls.append(newUrl)  
            print(list_urls) 
            findLinks(newUrl) # recursion
        else:
            if(link['href'].endswith('.zip')):
                target_path = link['href']
                response = requests.get(list_urls[-1] + link['href'], stream=True)
                if response.status_code == 200:
                    with open(target_path, 'wb') as f:
                        f.write(response.raw.read())
                
                # zip_file = ZipFile(list_urls[-1] + link['href'])         
                # temp_df = [open_complex_file(zip_file, text_file.filename)
                # for text_file in zip_file.infolist()
                # if text_file.filename.endswith('.csv')]
                # list_df.append(temp_df)

startUrl = "http://data.cquest.org/inpi_rncs/imr/stock/2017/"
findLinks(startUrl)

http://data.cquest.org/inpi_rncs/imr/stock/2017/05/
['http://data.cquest.org/inpi_rncs/imr/stock/2017/05/']
http://data.cquest.org/inpi_rncs/imr/stock/2017/05/04/
['http://data.cquest.org/inpi_rncs/imr/stock/2017/05/', 'http://data.cquest.org/inpi_rncs/imr/stock/2017/05/04/']


In [ ]:
import os
path2data=os.getcwd()

In [ ]:
path2data

In [ ]:
import functions as fc
list_df_rep = fc.import_all_rep(path2data + "/", "*.zip")

In [ ]:
len(list_df_rep)

In [ ]:
sys.getsizeof(list_df_rep)

In [ ]:
df_final = pd.concat(list_df_rep)

In [ ]:
df_final.shape # 9 504 907, 48 variables

In [ ]:
import s3fs
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
    
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET_OUT = "radjerad"
FILE_KEY_OUT_S3 = "inpi/" + "data_rep_2017.csv" 
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
    df_final.to_csv(file_out, index=False, sep=";", encoding="utf-8")

In [ ]:
list_df_pm = fc.import_all_pm(path2data + "/", "*.zip")

In [ ]:
len(list_df_pm)

In [ ]:
df_final_pm = pd.concat(list_df_pm)

In [ ]:
df_final_pm.shape # 4 296 552

In [ ]:
import s3fs
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
    
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET_OUT = "radjerad"
FILE_KEY_OUT_S3 = "inpi/" + "data_pm_2017.csv" 
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
    df_final_pm.to_csv(file_out, index=False, sep=";", encoding="utf-8")

In [ ]:
def export_2_minio(year, path2output, df_final):
    S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
    fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
    BUCKET_OUT = "radjerad"
    FILE_KEY_OUT_S3 = "inpi/" + year + "/" + path2output 
    FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

    with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
        df_final.to_csv(file_out)
    return None


In [ ]:
import pandas as pd
li = []
import s3fs
for i in range(len(list_df)):
    print(i)
    temp = pd.concat(list_df[i])
    print(temp.shape)
    temp = temp.reset_index()
    path2output = temp.loc[0,"source"][0:16]
    print(path2output)
    #temp.to_csv(path2output, sep=";")
    export_2_minio("2017", path2output + ".csv", temp)
    # temp.to_csv(path2output + ".csv", sep=";")
    # li.append(temp)
